Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [40]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 60
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)

    return tf.matmul(relu, weights3) + biases3
  
  
  def build_network(dataset):
    return logistic_regression_network(dataset)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 100.166733
Minibatch accuracy: 10.9%
Validation accuracy: 21.7%
Minibatch loss at step 500: 1.421501
Minibatch accuracy: 73.4%
Validation accuracy: 73.2%
Minibatch loss at step 1000: 0.662782
Minibatch accuracy: 82.8%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.835727
Minibatch accuracy: 73.4%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 1.233003
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 0.958488
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 0.892666
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Test accuracy: 87.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 60
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)

    return tf.matmul(relu, weights3) + biases3
  
  
  def build_network(dataset):
    return logistic_regression_network(dataset)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 10
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 92.054413
Minibatch accuracy: 6.2%
Validation accuracy: 24.7%
Minibatch loss at step 500: 0.000044
Minibatch accuracy: 100.0%
Validation accuracy: 45.5%
Minibatch loss at step 1000: 0.000024
Minibatch accuracy: 100.0%
Validation accuracy: 45.5%
Minibatch loss at step 1500: 0.000017
Minibatch accuracy: 100.0%
Validation accuracy: 45.5%
Minibatch loss at step 2000: 0.000013
Minibatch accuracy: 100.0%
Validation accuracy: 45.6%
Minibatch loss at step 2500: 0.000010
Minibatch accuracy: 100.0%
Validation accuracy: 45.6%
Minibatch loss at step 3000: 0.000009
Minibatch accuracy: 100.0%
Validation accuracy: 45.6%
Test accuracy: 50.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 60
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset, include_dropout):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)
    
    if include_dropout:
      first_level_output = tf.nn.dropout(relu, 0.45)
    else:
      first_level_output = relu

    return tf.matmul(first_level_output, weights3) + biases3
  
  
  def build_network(dataset, include_dropout=False):
    return logistic_regression_network(dataset, include_dropout)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset, True)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 10
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 142.005264
Minibatch accuracy: 9.4%
Validation accuracy: 24.6%
Minibatch loss at step 500: 0.330002
Minibatch accuracy: 88.3%
Validation accuracy: 51.2%
Minibatch loss at step 1000: 0.153478
Minibatch accuracy: 96.9%
Validation accuracy: 52.1%
Minibatch loss at step 1500: 0.264147
Minibatch accuracy: 92.2%
Validation accuracy: 52.4%
Minibatch loss at step 2000: 0.099850
Minibatch accuracy: 97.7%
Validation accuracy: 52.7%
Minibatch loss at step 2500: 0.107487
Minibatch accuracy: 96.9%
Validation accuracy: 52.8%
Minibatch loss at step 3000: 0.111145
Minibatch accuracy: 96.9%
Validation accuracy: 52.4%
Test accuracy: 58.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 1024
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset, include_dropout):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)
    
    if include_dropout:
      first_level_output = tf.nn.dropout(relu, 0.45)
    else:
      first_level_output = relu

    return tf.matmul(first_level_output, weights3) + biases3
  
  
  def build_network(dataset, include_dropout=False):
    return logistic_regression_network(dataset, include_dropout)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset, True)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 564.374146
Minibatch accuracy: 7.8%
Validation accuracy: 25.9%
Minibatch loss at step 500: 28.821896
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 13.150995
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 11.787013
Minibatch accuracy: 71.9%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 19.607998
Minibatch accuracy: 73.4%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 12.499114
Minibatch accuracy: 64.1%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 8.117805
Minibatch accuracy: 68.0%
Validation accuracy: 79.3%
Minibatch loss at step 3500: 2.118697
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 4000: 3.427001
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 4500: 5.553846
Minibatch accuracy: 75.8%
Validation accuracy: 79.5%
Minibatch loss at step 5000: 3.608072
Minibatch accuracy: 75.8%
Validation a

---
Try reducing learning rate
---

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 1024
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset, include_dropout):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)
    
    if include_dropout:
      first_level_output = tf.nn.dropout(relu, 0.45)
    else:
      first_level_output = relu

    return tf.matmul(first_level_output, weights3) + biases3
  
  
  def build_network(dataset, include_dropout=False):
    return logistic_regression_network(dataset, include_dropout)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset, True)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 492.846252
Minibatch accuracy: 16.4%
Validation accuracy: 32.6%
Minibatch loss at step 500: 38.930229
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 15.173672
Minibatch accuracy: 75.8%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 10.792430
Minibatch accuracy: 64.1%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 14.188278
Minibatch accuracy: 71.1%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 13.016068
Minibatch accuracy: 66.4%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 8.466230
Minibatch accuracy: 71.1%
Validation accuracy: 80.2%
Minibatch loss at step 3500: 2.305134
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 4000: 4.875748
Minibatch accuracy: 78.1%
Validation accuracy: 80.5%
Minibatch loss at step 4500: 3.516867
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 5000: 3.569818
Minibatch accuracy: 71.9%
Validation 

In [17]:
# Try reducing learning rate even further

batch_size = 128

graph = tf.Graph()
with graph.as_default():
  
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Level one
  
  num_internal_states = 1024
  
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_internal_states]))
  biases1 = tf.Variable(tf.zeros([num_internal_states]))
  
  # Variables. Level two. These are only 10x10 because that's what fits
  weights2 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_internal_states]))
  biases2 = tf.Variable(tf.zeros([num_internal_states]))
  
    
  # Variables. Level two. These are only 10x10 because that's what fits
  # Variables must be defined here and not in a function, otherwise they won't be changes/reused
  weights3 = tf.Variable(
    tf.truncated_normal([num_internal_states, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Return the logits with some input dataset
  def logistic_regression_network(dataset, include_dropout):
    first_level = tf.matmul(dataset, weights1) + biases1
    relu = tf.nn.relu(first_level)
    
    if include_dropout:
      first_level_output = tf.nn.dropout(relu, 0.45)
    else:
      first_level_output = relu

    return tf.matmul(first_level_output, weights3) + biases3
  
  
  def build_network(dataset, include_dropout=False):
    return logistic_regression_network(dataset, include_dropout)
  
  
  # Training computation.
  logits = build_network(tf_train_dataset, True)
  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500/8, 0.96)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    build_network(tf_valid_dataset)
  )
  test_prediction = tf.nn.softmax(
    build_network(tf_test_dataset)
  )
################################
# Run the system
################################

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction],
      feed_dict=feed_dict
    )
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 541.762207
Minibatch accuracy: 10.2%
Validation accuracy: 20.6%
Minibatch loss at step 500: 26.479359
Minibatch accuracy: 73.4%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 15.068511
Minibatch accuracy: 74.2%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 9.690345
Minibatch accuracy: 68.8%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 8.264042
Minibatch accuracy: 76.6%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 7.172189
Minibatch accuracy: 72.7%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 3.951151
Minibatch accuracy: 71.9%
Validation accuracy: 82.4%
Minibatch loss at step 3500: 2.763656
Minibatch accuracy: 78.1%
Validation accuracy: 82.8%
Minibatch loss at step 4000: 1.430373
Minibatch accuracy: 80.5%
Validation accuracy: 83.0%
Minibatch loss at step 4500: 2.751059
Minibatch accuracy: 80.5%
Validation accuracy: 83.2%
Minibatch loss at step 5000: 2.189981
Minibatch accuracy: 77.3%
Validation acc